# Aplicación de ML para predecir ubicación de nuevos Puntos de reciclaje
## Ciudad de Mendoza

### GRUPO 8
* Benjamin Berger
* Bruno Florio Sehor
* Gerardo Graña
* Hebe Munini

Presentación de detalles y resultados finales
https://docs.google.com/presentation/d/1sop7FFY0EzIWnD0_ExcNTRC9M86hNef1z3XeX2xgchQ/edit?usp=sharing

## Importación de librerías

In [1]:
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix

from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

## Preparación de los datos

In [ ]:
# Conexión a datos

PVR_observaciones_375 = pd.read_csv('../Datasets/PVR_observaciones_promedio_cutoff375.csv')
PVR_DAM_geodata = pd.read_csv('../Datasets/PVR_DAM_geodata.csv', delimiter= ';')

* **PVR_DAM_geodata:** contiene todos los features que entrenarán a los modelos. Dentro de esta tabla, existe la columna **punto_tipo.** Sobre la misma se aplicará un filtrado que permitirá segmentar el dataframe en 2 conjuntos. El primer conjunto (=! 'Intesrsección') contiene todos los puntos que entrenarán al modelo. El segundo conjunto (= 'Intersección') se utilizará con el modelo en producción como prototipo del proyecto.

* **PVR_observaciones:** contiene las etiquetas a predecir por los modelos. Estas representan una valoración de los Puntos de Reciclaje como 'aptos' o 'no aptos' según observaciones de campo. PVR_observaciones será parte de un proceso join con PVR_geodata para la construcción del dataframe PRV_train_test.

In [3]:
def Mergeador_data (data2, data1):
    
    # Guarda variables en forma global, para llamarlas en cualquier momento
    global PVR_train_test, DAM_to_predict, PVR_train_test_copy
    
    # La funcion elimina los puntos del tipo intersección, además guarda las intersecciones para luego hacer predicciones
    DAM_to_predict = data1[data1['punto_tipo'] == 'Intersercción']
    PVR_geodata = data1.drop(data1[data1['punto_tipo'] == 'Intersercción'].index)
    
    PVR_train_test = pd.merge(data2, data1 , how="left", on=["ID"]) 
    
    PVR_train_test.dropna(axis=0, inplace= True)
    
    # Copia del df con features de interés para representación en interfaz de usuario final (ID, index, x, y, punto_tipo)
    PVR_train_test_copy = PVR_train_test.copy()
    PVR_train_test.drop(['ID', 'index','x','y', 'punto_tipo', 'radio_censal'], axis=1, inplace= True)  
    
    PVR_train_test = pd.get_dummies(PVR_train_test, drop_first=True)

In [4]:
# Data merge, construcción de train_test

Mergeador_data(PVR_observaciones_375, PVR_DAM_geodata)

Ahora invocamos la función y creado el dataset hagamos otra que se quede con los mejores features, splitee los datos y nos guarde el dataset para entrenar.

In [9]:
def Preparando_data (data1):
    
    global x_train, x_test, y_train, y_test
        
    # Train test split 
    X = PVR_train_test[PVR_train_test.columns.difference(['valoracion'])]
    y = PVR_train_test['valoracion']
    x_train, x_test, y_train, y_test = train_test_split(X,y, stratify= y, test_size= 0.3, random_state= 9)
    
    # Estandarización [OFF]
    # x_train_std, x_test_std = MinMaxScaler(x_train, x_test)
    
    print('La distribucion de etiquetas es: ', y_train.value_counts() / y_train.value_counts().sum())

In [10]:
Preparando_data(PVR_train_test)

La distribucion de etiquetas es:  1.0    0.613636
0.0    0.386364
Name: valoracion, dtype: float64


## Entrenamiento y optimización de modelos predictivos

### Bagging

In [ ]:
classifier = {
    'KNeighbors': KNeighborsClassifier(),
    'LinearSCV': LinearSVC(),
    'SVC': SVC(),
    'SGDC': SGDClassifier(),
    'DecisionTree': DecisionTreeClassifier()
}

for name, estimator in classifier.items():
    bag_class = BaggingClassifier(
        base_estimator=estimator, n_estimators=5).fit(x_train, y_train)
    bag_pred = bag_class.predict(x_test)

    print('Accuracy Bagging with {}:'.format(
        name), accuracy_score(bag_pred, y_test))
    print('')


### Boosting

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=41, shuffle=True)

params = {
    "n_estimators": [50, 100, 500],
    "max_depth": [1, 2, 3, 4],
    "learning_rate": [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1],
}

params2 = {
    "n_estimators": [50, 100, 500],
    "learning_rate": [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1],
}

# XG Boost

model_xg = XGBClassifier(n_jobs=-1, use_label_encoder=False)

xgb = GridSearchCV(model_xg, param_grid=params,
                   cv=cv, verbose=1, n_jobs=-1)

xgb.fit(x_train, y_train)

# Gradient Boosting

model_gb = GradientBoostingClassifier()

gb = GridSearchCV(model_gb, param_grid=params, cv=cv, verbose=1, n_jobs=-1)

gb.fit(x_train, y_train)

# Ada Boost

model_adb = AdaBoostClassifier()

adb = GridSearchCV(model_adb, param_grid=params2,
                   cv=cv, verbose=1, n_jobs=-1)

adb.fit(x_train, y_train)


## Evaluación de modelos

### Mejores parámetros y matrices de confusión

In [13]:
classifiers = [AdaBoostClassifier(learning_rate=1, n_estimators=500),
               GradientBoostingClassifier(
                   learning_rate=0.75, max_depth=1, n_estimators=100),
               XGBClassifier(learning_rate=0.75, max_depth=3, n_estimators=50)]
for cls in classifiers:
    cls.fit(x_train, y_train)


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

for cls, ax in zip(classifiers, axes.flatten()):
    plot_confusion_matrix(cls,
                          x_test,
                          y_test,
                          ax=ax,
                          cmap='Blues',
                          display_labels=['No Apto', 'Apto'])
    ax.title.set_text(type(cls).__name__)
plt.tight_layout()
plt.show()


### Rendimiento para los modelos con cross validation

In [15]:
def evaluar_rendimiento(modelo, nombre, X, y, cv):
    s = cross_val_score(modelo, X, y, cv=cv, n_jobs=-1)
    print("Rendimiento de {}:\t{:0.3} ± {:0.3}".format(
        nombre, s.mean().round(3), s.std().round(3)))

In [16]:
evaluar_rendimiento(xgb.best_estimator_,"XG Boost", x_train, y_train, cv)

Rendimiento de XG Boost:	0.728 ± 0.051


**Mejor rendimiento**

In [17]:
evaluar_rendimiento(gb.best_estimator_,
                    "Gradient Boosting", x_train, y_train, cv)


Rendimiento de Gradient Boosting:	0.753 ± 0.106


In [18]:
evaluar_rendimiento(adb.best_estimator_,"Ada Boost", x_train, y_train, cv)

Rendimiento de Ada Boost:	0.683 ± 0.126


## Predicciones

Se eliminan los features que no necesitamos para hacer las predicciones sobre el dataframe DAM_to_predict que contiene los cruces de calles de toda la ciudad.

### Preparación de los datos

In [ ]:
# Selección de features y dummies

DAM_to_predict.dropna(axis=0, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Especial Parque Gral. San Marti­n'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Especial UNCuyo'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Residencial de uso controlado'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Residencial piedemonte'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Area de Seguridad'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Especial Ex-Aeroparque'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Especial Ejercito'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Especial Club San Ceferino'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Natural de amortiguacion'].index, inplace=True)
DAM_to_predict.drop(DAM_to_predict[DAM_to_predict['microzonificacion']=='Zona de inundacion de dique'].index, inplace=True)

DAM_to_predict_final = DAM_to_predict.copy()
DAM_to_predict.drop(['ID', 'index','x','y', 'punto_tipo','radio_censal'], axis=1, inplace=True)
data = pd.get_dummies(DAM_to_predict, drop_first=True)

### Predicciones

In [20]:
Predicciones = gb.best_estimator_.predict(data)

print(pd.DataFrame(Predicciones))
print(pd.DataFrame(Predicciones).value_counts())

        0
0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
...   ...
1843  1.0
1844  1.0
1845  0.0
1846  1.0
1847  1.0

[1848 rows x 1 columns]
0.0    1014
1.0     834
dtype: int64


/Users/benjamin/anaconda3/envs/dhdsblend2021/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [21]:
# Incorporación de las predicciones al dataframe final

DAM_to_predict_final['clasificacion'] = Predicciones
DAM_to_predict_final['clasificacion'] = Predicciones

## Output API

In [22]:
# Preparación de datos de salida para API web
 
DAM_to_predict_final.to_csv("../Data/PVR_DAM_all.csv", index=False, encoding='utf-8-sig')